In [1]:
from nltk.stem import WordNetLemmatizer
import nltk
import string as str
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
punctuation_signs=list('''!()-[]{};:'"\,<>./?#^_~''')
wordnet_lemmatizer = WordNetLemmatizer()
stop_words =list(stopwords.words('english')) + punctuation_signs
category_codes = {
    'Business': 0,
    'Entertainment': 1,
    'Politics': 2,
    'Sport': 3,
    'Tech': 4
}

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssm392/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ssm392/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ssm392/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
## NLPppppppppp
import pandas as pd

In [3]:
filepath = r'/Users/ssm392/Desktop/IV Project/data/df_trainedSet.jsonl'
with open(filepath) as json_data:
    df_process_bbc = pd.read_json(json_data,lines=True)

In [4]:
df_process_bbc = pd.read_json(filepath, lines=True)

In [5]:
type(df_process_bbc)

pandas.core.frame.DataFrame

In [6]:
df_process_bbc.head(2)

,Category_code,Content_Parsed,News_length
0,Business,ad sales boost time warner profit quarterly pr...,2560
1,Business,dollar gain greenspan speech dollar hit highes...,2252


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_process_bbc['Content_Parsed'], 
                                                    df_process_bbc['Category_code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [10]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


In [11]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Business' category:
  . Most correlated unigrams:
. five
. first
. firm
. final
. years
  . Most correlated bigrams:
. last year
. year old

# 'Entertainment' category:
  . Most correlated unigrams:
. five
. first
. firm
. final
. years
  . Most correlated bigrams:
. last year
. year old

# 'Politics' category:
  . Most correlated unigrams:
. five
. first
. firm
. final
. years
  . Most correlated bigrams:
. last year
. year old

# 'Sport' category:
  . Most correlated unigrams:
. five
. first
. firm
. final
. years
  . Most correlated bigrams:
. last year
. year old

# 'Tech' category:
  . Most correlated unigrams:
. five
. first
. firm
. final
. years
  . Most correlated bigrams:
. last year
. year old



In [12]:
print(type(labels_train))
print(type(features_test))
print(type(labels_test))
print(type(tfidf))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'sklearn.feature_extraction.text.TfidfVectorizer'>


In [13]:
idf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
idf = pd.DataFrame(columns=['idf']).from_dict(dict(idf), orient='index')
idf.columns = ['idf']

In [14]:
#idf

In [15]:
x = tfidf.get_feature_names()
print(len(x))

300


In [47]:
#Export = idf.to_json(r'C:\Users\mouni\OneDrive\Desktop\Info VIz\trained_files\tfidf.jsonl',orient='records', lines=True)

# SVC Hyper parameter model prediction:

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
svc_0 =svm.SVC(random_state=8)

print('Parameters currently in use:\n')
pprint(svc_0.get_params())

Parameters currently in use:

{'C': 1.0,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'auto_deprecated',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 8,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [18]:
print(features_train.shape)
print(features_test.shape)

(1891, 300)
(334, 300)


In [19]:
C = [.0001, .001, .01, .1]
degree = [3, 4, 5]
gamma = [1, 10, 100]
probability = [True]

param_grid = [
  {'C': C, 'kernel':['linear'], 'probability':probability},
  {'C': C, 'kernel':['poly'], 'degree':degree, 'probability':probability},
  {'C': C, 'kernel':['rbf'], 'gamma':gamma, 'probability':probability}
]

# Create a base model
svc = svm.SVC(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=svc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(features_train, labels_train)

Fitting 3 folds for each of 28 candidates, totalling 84 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid th

/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ssm392/anaconda3/lib/python3.7/site-packages/

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=8, test_size=0.33, train_size=None),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=8,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['linear'], 'probability': [True]}, {'C': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['poly'], 'degree': [3, 4, 5], 'probability': [True]}, {'C': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf'], 'gamma': [1, 10, 100], 'probability': [True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [20]:
print("The best hyperparameters from Grid Search are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

The best hyperparameters from Grid Search are:
{'C': 0.1, 'kernel': 'linear', 'probability': True}

The mean accuracy of a model with these hyperparameters is:
0.9482666666666667


In [26]:
best_svc = grid_search.best_estimator_
best_svc

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=8,
  shrinking=True, tol=0.001, verbose=False)

# Model Fit and performance

In [27]:
best_svc.fit(features_train, labels_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=8,
  shrinking=True, tol=0.001, verbose=False)

In [28]:
svc_pred = best_svc.predict(features_test)
#svc_pred = best_svc.predict_proba(features_test)

In [29]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(labels_train, best_svc.predict(features_train)))

The training accuracy is: 
0.9592808038075092


In [30]:
print("The test accuracy is: ")
print(accuracy_score(labels_test, svc_pred))

The test accuracy is: 
0.9341317365269461


In [31]:
print("Classification report")
print(classification_report(labels_test,svc_pred))

Classification report
               precision    recall  f1-score   support

     Business       0.86      0.98      0.91        81
Entertainment       0.96      0.94      0.95        49
     Politics       0.97      0.88      0.92        72
        Sport       0.99      0.99      0.99        72
         Tech       0.93      0.88      0.91        60

    micro avg       0.93      0.93      0.93       334
    macro avg       0.94      0.93      0.93       334
 weighted avg       0.94      0.93      0.93       334



In [32]:
base_model = svm.SVC(random_state = 8)
base_model.fit(features_train, labels_train)
accuracy_score(labels_test, base_model.predict(features_test))

/Users/ssm392/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.2155688622754491

In [33]:
best_svc.fit(features_train, labels_train)
accuracy_score(labels_test, best_svc.predict(features_test))

0.9341317365269461

In [34]:
d = {
     'Model': 'SVM',
     'Training Set Accuracy': accuracy_score(labels_train, best_svc.predict(features_train)),
     'Test Set Accuracy': accuracy_score(labels_test, svc_pred)
}

df_models_svc = pd.DataFrame(d, index=[0])

In [35]:
#with open(r'C:\Users\mouni\OneDrive\Desktop\Info VIz\trained_files\best_svc.pickle', 'wb') as output:
 #   pickle.dump(best_svc, output)

In [36]:
predictions = best_svc.predict(features_test)

In [37]:
# Indexes of the test set
index_X_test = X_test.index
# We get them from the original df
df_test = df_process_bbc.loc[index_X_test]
df_test['Category_Predicted'] = predictions
for col in df_test.columns: 
    print(col)
df_test.head(5)

Category_code
Content_Parsed
News_length
Category_Predicted


,Category_code,Content_Parsed,News_length,Category_Predicted
1691,Sport,ireland call uncapped campbell ulster scrum-ha...,2431,Sport
1103,Politics,gurkhas help tsunami victims britain offer sen...,1793,Business
477,Business,egypt israel seal trade deal sign thaw relatio...,1716,Business
197,Business,cairn share new oil find share cairn energy ju...,1658,Business
475,Business,saudi ncci share soar share saudi arabia natio...,1306,Business


In [38]:
category_names = {
    0: 'Business',
    1: 'Entertainment',
    2: 'Politics',
    3: 'Sport',
    4: 'Tech'
}
df_test = df_test.replace({'Category_Predicted':category_names})
df_test.head(5)

,Category_code,Content_Parsed,News_length,Category_Predicted
1691,Sport,ireland call uncapped campbell ulster scrum-ha...,2431,Sport
1103,Politics,gurkhas help tsunami victims britain offer sen...,1793,Business
477,Business,egypt israel seal trade deal sign thaw relatio...,1716,Business
197,Business,cairn share new oil find share cairn energy ju...,1658,Business
475,Business,saudi ncci share soar share saudi arabia natio...,1306,Business


In [39]:
df_misclassified = df_test[df_test['Category_code'] != df_test['Category_Predicted']]
print(len(df_test))
print(len(df_misclassified))
df_misclassified.head(3)

334
22


,Category_code,Content_Parsed,News_length,Category_Predicted
1103,Politics,gurkhas help tsunami victims britain offer sen...,1793,Business
1880,Tech,half-life 2 sweep bafta award pc first person ...,1379,Entertainment
2137,Tech,junk e-mail relentless rise spam traffic 40 % ...,1532,Business


# MIllion test data classification

In [40]:
import pandas as pd
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import bulk, streaming_bulk
import uuid
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}],timeout=30, max_retries=10, retry_on_timeout=True)
import json
import time
from pandas.io.json import json_normalize

In [41]:
nlpdf = pd.DataFrame()

start = time.time()
page = es.search(
    index='ppalldataindex',
    scroll='40m',
    size=10000,
    _source=["NLPid","Content_Parsed","News_length"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

nlpdf = nlpdf.append(json_normalize(page['hits']['hits']))
i=1
while (scroll_size > 0):
    #print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '40m')
    nlpdf = nlpdf.append(json_normalize(page['hits']['hits']))
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
end = time.time() - start
print(end)
print(len(nlpdf))


276.13541984558105
1000000


In [43]:
len(nlpdf)

1000000

In [44]:
nlpdf.head(1)

,_id,_index,_score,_source.Content_Parsed,_source.NLPid,_source.News_length,_type
0,s_fcJmwBMJThRIIz12OB,ppalldataindex,1.0,two thousand ... six hundred sixty-six dollaaa...,946b2ba4-832b-4501-b802-8b4955dcc129,754,doc


In [45]:
features_test1 = tfidf.transform(nlpdf["_source.Content_Parsed"]).toarray()
print(features_test1.shape)


(1000000, 300)


In [46]:
svc_pred = best_svc.predict(features_test1)

In [47]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category
def predict_from_text(num,text,nl):
    
    # Predict using the input model
    X_series = pd.Series(text)
    features_test2 = tfidf.transform(X_series).toarray()
    prediction_svc = best_svc.predict(features_test2)
    prediction_svc_proba = best_svc.predict_proba(features_test2)
    
    # Return result
    #category_svc = get_category_name(prediction_svc)
    d_series = {}
    d_series["Content"] = text
    d_series["id"]= num
    d_series["newslength"]= nl
    if(prediction_svc_proba.max()*100 <= 65):
        d_series["Category_Predicted"] = "Others"
    else:
        #print(prediction_svc[0])
        d_series["Category_Predicted"] = prediction_svc[0]
    #print(d_series["Category_Predicted"])
    return d_series

In [48]:
import swifter
#df_series_100 =[]
#X_test_series= nlpdf["_source.Content_Parsed"]
df_final = nlpdf[["_source.NLPid","_source.Content_Parsed","_source.News_length"]].swifter.apply(lambda row: predict_from_text(row["_source.NLPid"],row["_source.Content_Parsed"],row["_source.News_length"]),axis = 1)
print(len(df_final))


/Users/ssm392/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



1000000


In [49]:
len(df_final)

1000000

In [51]:
type(df_final)

pandas.core.series.Series

In [52]:
import pickle
with open(r'/Users/ssm392/Desktop/IV Project/data/nlpdataSeries.pickle', 'wb') as output:
    pickle.dump(df_final, output)
     

In [54]:
#nlpdf_final = df_final.swifter.apply(pd.Series)

## Store back to ES DB

In [1]:
import pandas as pd
postnlp_df = pd.read_pickle(r'/Users/ssm392/Desktop/IV Project/data/nlpdataSeries.pickle')


In [2]:
type(postnlp_df)

pandas.core.series.Series

In [3]:
import swifter
df = postnlp_df.swifter.apply(pd.Series)

/Users/ssm392/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.head(10)

,Content,id,newslength,Category_Predicted
0,two thousand ... six hundred sixty-six dollaaa...,946b2ba4-832b-4501-b802-8b4955dcc129,754,Others
1,unleash entrepreneurial spirit towards own tow...,96e1f93e-3182-4754-b24a-0ad5bf2f6631,4178,Tech
2,'re one country leaders probably n't get much ...,977bff20-fde0-45d4-bdf7-c2e69e0928d0,928,Entertainment
3,athlone gaa player owe life quick action medic...,40ed5fcd-8cd9-47ab-b363-548866b76221,2429,Sport
4,emerson electric nyse emr open bullishly pivot...,685c537c-89dd-4e1b-96e6-e40275c8d9c4,232,Sport
5,narrow five-inch scar across aries merritt sto...,66c2c3bb-613d-47e1-908e-609990faf292,3892,Sport
6,excavate bone tree root marion dowd archaeolog...,0c95a0ec-5bad-4009-92b2-89287c1af449,1826,Entertainment
7,15 relax lee * buddy rich charlie parker curle...,e74554f4-49dd-48b1-9d87-2af47ff10da9,3564,Entertainment
8,comment 0 loss many players summer reflect pit...,7ae1d967-a556-46b4-9a80-208f012da9f0,3636,Sport
9,doubt drink coasters remind adventurous days f...,ea1a7526-6624-486e-8bcf-cb186201f27e,1374,Tech


In [10]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
#import uuid
#import json_lines, os, json
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}])

In [11]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

use_these_keys = ['Content', 'id', 'newslength','Category_Predicted']

def bulk_json_data(_index, doc_type): 
    for index, document in df.iterrows():
        yield {
            "_index": _index,
            "_type": doc_type,
            "_source": filterKeys(document)
        }

In [12]:
success, failed = 0, 0

# list of errors to be collected is not stats_only
errors = []
i=1
for ok, item in streaming_bulk(es, bulk_json_data('postnlpdata','doc'), chunk_size=1000, max_retries=5,initial_backoff=2, max_backoff=600, request_timeout=3600):
    # go through request-reponse pairs and detect failures
    if not ok:
        if not stats_only:
            errors.append(item)
        failed += 1
    else:
        success += 1
        #print(success)
#print(success)

In [18]:
## Merge dfs

import pandas as pd
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import bulk, streaming_bulk
import uuid
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}],timeout=30, max_retries=10, retry_on_timeout=True)
import json
import time
from pandas.io.json import json_normalize


In [25]:
alldf = pd.DataFrame()

start = time.time()
page = es.search(
    index='alldataindex',
    scroll='40m',
    size=10000,
    #_source=["id","content"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

alldf = alldf.append(json_normalize(page['hits']['hits']))
i=1
while (scroll_size > 0):
    #print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '40m')
    alldf = alldf.append(json_normalize(page['hits']['hits']))
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])
end = time.time() - start
print(end)
print(len(alldf))


503.10945200920105
1000000


In [26]:
df.head(2)  #nlp df

,id,newslength,Category_Predicted
0,946b2ba4-832b-4501-b802-8b4955dcc129,754,Others
1,96e1f93e-3182-4754-b24a-0ad5bf2f6631,4178,Tech


In [35]:
alldf.head(2)

,Content,id,media-type,datePublished,source,title
0,TWO THOUSAND... SIX HUNDRED AND SIXTY-SIX DOLL...,946b2ba4-832b-4501-b802-8b4955dcc129,Blog,2015-09-11T16:32:45Z,Indiegogo: Announcements,Fri Sep 11 '15 Announcement from Women Write A...
1,Unleashing your entrepreneurial spirit towards...,96e1f93e-3182-4754-b24a-0ad5bf2f6631,News,2015-09-26T04:00:00Z,Hamilton Community News,Another memo to Alex Johnstone


In [ ]:
#Delete columns: 
del df['Content'] 

In [32]:
del alldf['_type'] #,'_index','_score','_type']

In [34]:
#Change column names : 
alldf.columns = ['Content','id','media-type','datePublished','source','title'] 

In [36]:
#Dataframe merge: 
df_final=pd.merge(alldf, df, how='inner', on='id') 

In [38]:
len(df_final) #.head(2)

1000000

In [41]:
df_final.head(2)

,Content,id,media-type,datePublished,source,title,newslength,Category_Predicted
0,TWO THOUSAND... SIX HUNDRED AND SIXTY-SIX DOLL...,946b2ba4-832b-4501-b802-8b4955dcc129,Blog,2015-09-11T16:32:45Z,Indiegogo: Announcements,Fri Sep 11 '15 Announcement from Women Write A...,754,Others
1,Unleashing your entrepreneurial spirit towards...,96e1f93e-3182-4754-b24a-0ad5bf2f6631,News,2015-09-26T04:00:00Z,Hamilton Community News,Another memo to Alex Johnstone,4178,Tech


In [49]:
import pickle
with open(r'/Users/ssm392/Desktop/IV Project/data/FinalData.pickle', 'wb') as output:
    pickle.dump(df_final, output)

KeyboardInterrupt: 

In [50]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
#import uuid
#import json_lines, os, json
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}])

In [53]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

use_these_keys = ['Content','id', 'media-type','datePublished','source','title','newslength','Category_Predicted']

def bulk_json_data(_index, doc_type): 
    for index, document in df_final.iterrows():
        yield {
            "_index": _index,
            "_type": doc_type,
            "_source": filterKeys(document)
        }

In [54]:
success, failed = 0, 0

# list of errors to be collected is not stats_only
errors = []
i=1
for ok, item in streaming_bulk(es, bulk_json_data('finalnewsdata','doc'), chunk_size=1000, max_retries=5,initial_backoff=2, max_backoff=600, request_timeout=3600):
    # go through request-reponse pairs and detect failures
    if not ok:
        if not stats_only:
            errors.append(item)
        failed += 1
    else:
        success += 1
        #print(success)
#print(success)